# Reappraisal Training For Linguistic Distancing and Emotion Regulation


## Setup
```bash
> pipenv shell  #Generates a new virtual environment based on Pipfile
> pipenv install # Installs the packages in Pipfile.lock (Use --dev) to also install dev packages
```
## Included Datasets
- LDHII 
- 
- Emobank
**Sources**
-  [Sentiment Analysis Text Classification Tutorial](https://www.youtube.com/watch?v=8N-nM3QW7O0)
- [Using Catalyst for Training Organization](https://github.com/catalyst-team/catalyst)



In [1]:

# Uncomment to add the src code to the working dir when on colab
# git init
# git config core.sparseCheckout true
# git remote add -f origin https://github.com/danielcpham/reappraisal-model.git
# echo "src/" > .git/info/sparse-checkout
# echo "poetry.lock" >> .git/info/sparse-checkout
#echo "pyproject.toml" >> .git/info/sparse-checkout 
# git checkout dev

# ! pip install transformers datasets 


Reinitialized existing Git repository in /Users/danielpham/Documents/code/reapp/.git/
fatal: remote origin already exists.
M	poetry.lock
M	pyproject.toml
M	src/LDHData.py
M	src/ReappModel.py
Already on 'dev'
Your branch is ahead of 'origin/dev' by 2 commits.
  (use "git push" to publish your local commits)

This repository is configured for Git LFS but 'git-lfs' was not found on your path. If you no longer wish to use Git LFS, remove this hook by deleting .git/hooks/post-checkout.



In [2]:
import os
import numpy as np
import pandas as pd
import torch
from datasets import ReadInstruction

# Enable GPU usage, if we can.
if torch.cuda.is_available():
    print("Enabling GPU usage")
    device = torch.device("cuda:0")
    print(device)
    IS_GPU = True
else:
    print("No GPU available, running on CPU")
    device = torch.device("cpu") # Note: macOS incompatible with NVIDIA GPUs
    IS_GPU = False
    

PRETRAINED_MODEL_NAME = 'distilbert-base-uncased-finetuned-sst-2-english'

No GPU available, running on CPU


### LDH Dataset Imports

In [3]:
from src.LDHData import LDHData

data = LDHData()
data.load_training_data()
data.load_eval_data()

Training data loaded from disk.
Evaluation data loaded from disk.


In [4]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained(PRETRAINED_MODEL_NAME)  
# Wrap tokenizer.
def tokenize(x):
    tokenized = tokenizer(x, add_special_tokens=True, padding="max_length", max_length=150)
    return tokenized
    
encoded_train = data.train_dataset['far'].map(
    lambda ds: tokenize(ds['response']), batched=True, batch_size=16
)
encoded_train.set_format(type='torch', output_all_columns=True)

Loading cached processed dataset at /Users/danielpham/Documents/code/reapp/src/training/far/cache-c09023214f11a34f.arrow


In [7]:
from transformers import DistilBertModel, TrainingArguments, Trainer
from src.ReappModel import ReappModel
# Define the parameters under which the model will be trained.
# By default, uses an AdamW optimizer w/ linear warmup.
model = ReappModel(DistilBertModel.from_pretrained(PRETRAINED_MODEL_NAME))

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,
    args=training_args,                  
    train_dataset=encoded_train,
    #eval_dataset=encoded_eval         
)

tensor([[ 0.0052,  0.1478],
        [-0.2810,  0.2687],
        [ 0.0080,  0.1550],
        [-0.0645, -0.0440],
        [-0.3149,  0.3754],
        [-0.0875,  0.0917],
        [-0.2683,  0.2093],
        [-0.1531,  0.2634],
        [-0.2345,  0.2897],
        [-0.1325,  0.2053],
        [-0.2943,  0.1872],
        [ 0.0488,  0.0393],
        [-0.1339, -0.0532],
        [-0.1767,  0.2387],
        [ 0.0352, -0.0639],
        [-0.0308,  0.1296]], grad_fn=<AddmmBackward>)


tensor(17.3436)